**Data Extraction**

In [ ]:
!pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

خواندن دیتا و قرار دادن در یک فایل CSV


In [36]:
adult = fetch_ucirepo(id=2)
df = adult.data.features.copy()
df["income"] = adult.data.targets["income"]
df.to_csv("adult.csv", index=False)

در این قسمت یک بررسی اولیه به داده ها می کنیم تا متوجه بشیم با چه نوع داده هایی مواجه هستیم

In [94]:
df = pd.read_csv("adult.csv")
#df.info()
#df.describe()
#df.isna().sum()

Pre Processing

In [95]:
df.drop(columns=["fnlwgt"], inplace=True)
df.drop(columns=["education"], inplace=True)

In [96]:
def auto_data_type(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        min_value = df[col].min()
        max_value = df[col].max()

        if pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].astype(np.float32)
        elif min_value >= 0:
            if max_value <= 255:
                df[col] = df[col].astype(np.uint8)
            elif max_value <= 65535:
                df[col] = df[col].astype(np.uint16)
            elif max_value <= 4294967295:
                df[col] = df[col].astype(np.uint32)
            else:
                df[col] = df[col].astype(np.uint64)
        else:
            if -128 <= min_value and max_value <= 127:
                df[col] = df[col].astype(np.int8)
            elif -32768 <= min_value and max_value <= 32767:
                df[col] = df[col].astype(np.int16)
            elif -2147483648 <= min_value and max_value <= 2147483647:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
auto_data_type(df)

پر کردن مقادیر خالی با مد آن ستون چون داده متنی داریم

In [ ]:
df.replace("?", np.nan, inplace=True)

df["workclass"].fillna(df["workclass"].mode()[0], inplace=True)
df["occupation"].fillna(df["occupation"].mode()[0], inplace=True)
df["native-country"].fillna(df["native-country"].mode()[0], inplace=True)


مپ کردن مقادیر ستون ها با اعداد برای کاهش حجم دیتا  (می توان ستون اصلی را هم  در صورت نیاز پاک کرد)

In [ ]:
df["sex"] = df["sex"].map({"Male": 0, "Female": 1})
df["income"] = df["income"].map({"<=50K": 0, ">50K": 1})
df["sex"] = df["sex"].astype(np.uint8)
df = df[df["income"].notna()]
df["income"] = df["income"].astype(np.uint8)


df["marital_status_code"] = df["marital-status"].map({
    "Married-civ-spouse": 0,
    "Married-spouse-absent": 1,
    "Never-married": 2,
    "Divorced": 3,
    "Separated": 4,
    "Widowed": 5
})
df["marital_status_code"] = df["marital_status_code"].fillna(-1).astype(np.int8)



df["race_code"] = df["race"].map({
    "White": 0,
    "Black": 1,
    "Asian-Pac-Islander": 2,
    "Amer-Indian-Eskimo": 3,
    "Other": 4
}).astype(np.uint8)

df["relationship_code"] = df["relationship"].map({
    "Husband": 0,
    "Not-in-family": 1,
    "Own-child": 2,
    "Unmarried": 3,
    "Wife": 4,
    "Other-relative": 5
}).astype(np.uint8)

df["workclass_code"] = df["workclass"].map({
    "Private": 0,
    "Self-emp-not-inc": 1,
    "Self-emp-inc": 2,
    "Federal-gov": 3,
    "Local-gov": 4,
    "State-gov": 5,
    "Without-pay": 6,
    "Never-worked": 7,
    "Unknown": 8
}).astype(np.uint8)

df["occupation_code"] = df["occupation"].map({
    "Tech-support": 0,
    "Craft-repair": 1,
    "Other-service": 2,
    "Sales": 3,
    "Exec-managerial": 4,
    "Prof-specialty": 5,
    "Handlers-cleaners": 6,
    "Machine-op-inspct": 7,
    "Adm-clerical": 8,
    "Farming-fishing": 9,
    "Transport-moving": 10,
    "Priv-house-serv": 11,
    "Protective-serv": 12,
    "Armed-Forces": 13,
    "Unknown": 14
})
df["occupation_code"] = df["occupation_code"].fillna(-1).astype(np.int8)

for col in ["occupation_code", "marital_status_code"]:
    df[col] = df[col].replace(-1, df[col].mode()[0])
    df[col]=df[col].astype(np.uint8)


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32561 entries, 0 to 32560
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   age                  32561 non-null  uint8 
 1   workclass            32561 non-null  object
 2   education-num        32561 non-null  uint8 
 3   marital-status       32561 non-null  object
 4   occupation           32561 non-null  object
 5   relationship         32561 non-null  object
 6   race                 32561 non-null  object
 7   sex                  32561 non-null  uint8 
 8   capital-gain         32561 non-null  uint32
 9   capital-loss         32561 non-null  uint16
 10  hours-per-week       32561 non-null  uint8 
 11  native-country       32561 non-null  object
 12  income               32561 non-null  uint8 
 13  marital_status_code  32561 non-null  uint8 
 14  race_code            32561 non-null  uint8 
 15  relationship_code    32561 non-null  uint8 
 16  workclass